LIMPIEZA DE DATOS

In [1]:
import numpy as np 
import pandas as pd
import json 
import ast

In [ ]:
# Carga los data
df = pd.read_csv('movies_dataset.csv')

In [3]:

#  Desanidar los campos 'belongs_to_collection' y 'production_companies'
df['belongs_to_collection'] = df['belongs_to_collection'].apply(lambda x: x['name'] if isinstance(x, dict) else np.nan)
df['production_companies'] = df['production_companies'].apply(lambda x: ', '.join([d['name'] for d in x]) if isinstance(x, list) else np.nan)

In [4]:

# Rellenar los nulos en 'revenue' y 'budget' con 0
df['revenue'] = df['revenue'].fillna(0)
df['budget'] = df['budget'].fillna(0)

In [5]:

# Elimina valores nulos en 'release_date'
df.dropna(subset=['release_date'], inplace=True)

In [6]:
# Formatea fechas y crear la columna 'release_year'
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce').dt.strftime('%Y-%m-%d')
df['release_year'] = pd.to_datetime(df['release_date'], errors='coerce').dt.year

In [7]:
# crea la columna 'return'

# Converti 'revenue' y 'budget' a valores numéricos, forzando los valores no numéricos a NaN
df['revenue'] = pd.to_numeric(df['revenue'], errors='coerce')
df['budget'] = pd.to_numeric(df['budget'], errors='coerce')

# Rellene valores nulos con 0
df['revenue'] = df['revenue'].fillna(0)
df['budget'] = df['budget'].fillna(0)


# Creo la columna 'return'
df['return'] = df.apply(lambda row: row['revenue'] / row['budget'] if row['budget'] > 0 else 0, axis=1)


In [ ]:
# Elimine las columnas no utilizadas
df.drop(columns=['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage', 'original_language', 'production_countries' , 'spoken_languages', 'runtime'], inplace=True)


In [9]:
# desanida belongs_to_collection
df['belongs_to_collection'] = df['belongs_to_collection'].apply(lambda x: x['name'] if isinstance(x, dict) else np.nan)

# Desanidar production_companies
df['production_companies'] = df['production_companies'].apply(lambda x: ', '.join([d['name'] for d in x]) if isinstance(x, list) else np.nan)


In [10]:

df['collection_name'] = df['belongs_to_collection'].apply(lambda x: x['name'] if isinstance(x, dict) else np.nan)

df['production_companies_names'] = df['production_companies'].apply(lambda x: ', '.join([d['name'] for d in x]) if isinstance(x, list) else np.nan)


In [11]:
# Asegúrate de que la columna 'release_date' está en formato de fecha
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')

# Filtra el DataFrame para conservar solo los datos a partir de 1990
df = df[df['release_date'].dt.year >= 1990].reset_index(drop=True)


In [12]:
# # Verificamos formato de fecha
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')

df = df[df['release_date'].dt.year >= 1994].reset_index(drop=True)

# Opcional: imprime la cantidad de datos para verificar
print(f"Cantidad de datos después de 1994: {len(df)}")


Cantidad de datos después de 1994: 27889


In [13]:
credits= pd.read_csv('credits.csv')

In [14]:
# Extraer solo los nombres de los actores en la columna 'cast'
credits['cast'] = credits['cast'].apply(
    lambda x: [actor['name'] for actor in ast.literal_eval(x) if 'name' in actor] if isinstance(x, str) else []
)

# Guardar el dataset actualizado
credits.to_csv('movies_transformed.csv', index=False)


In [15]:
# Extraer solo los nombres de los directores en la columna 'crew'
credits['crew'] = credits['crew'].apply(
    lambda x: [director['name'] for director in ast.literal_eval(x) if 'name' in director] if isinstance(x, str) else []
)


credits.to_csv('movies_transformed.csv', index=False)

In [21]:
#unimos credits y movies para hacer los endpoint de actores y directores
movies_transformed = pd.read_csv("movies_transformed.csv")
movies_dataset_transformed = pd.read_csv("movies_dataset_transformed.csv")

movies_transformed['id'] = movies_transformed['id'].astype(str)
movies_dataset_transformed['id'] = movies_dataset_transformed['id'].astype(str)

# usamos inner join para mantener solo los id que coinciden
merged_data = movies_transformed.merge(movies_dataset_transformed, on="id", how="inner")

merged_data.to_csv("movies_completo.csv", index=False)

In [ ]:
merged_data.drop(columns=['original_language', 'production_countries' , 'spoken_languages', 'runtime'], inplace=True)

In [27]:
merged_data.drop(columns=['status', 'tagline' , 'production_companies_names'], inplace=True)

In [29]:
merged_data.drop(columns=['collection_name'], inplace=True)

In [ ]:
# Convertir la columna 'release_date' a formato datetime
merged_data['release_date'] = pd.to_datetime(merged_data['release_date'], errors='coerce')

# Filtrar películas entre los años 1994 y 2012
merged_data = merged_data[(merged_data['release_date'].dt.year >= 1994) & 
                          (merged_data['release_date'].dt.year <= 2012)].reset_index(drop=True)

# Imprimir la cantidad de datos después del filtrado
print(f"Cantidad de datos después del filtrado por rango de años: {len(merged_data)}")


In [32]:
merged_data.to_csv("movies_completo.csv", index=False)